In [1]:
from HinetPy import Client, win32
# %matplotlib widget

import matplotlib.pyplot as plt
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
import numpy as np
from obspy.geodetics import base
from obspy.taup import TauPyModel
from scipy import stats
import glob
from obspy import read
# %matplotlib widget
#InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'HinetPy'

In [ ]:
# Download data from iris
client = Client("yuqingxie","938512")
client.select_stations('0101', ['N.MKJH']) #0101 N.MKJH 37.4703 140.7227 510.0
#client.select_stations('0101', latitude=36, longitude=139,minradius=0, maxradius=3
# stations = client.get_station_list('0101')
# for station in stations:
#      print(station)
# startdate = datetime(2010, 1, 1)
# client.get_arrivaltime(startdate, 5)

#'0101' is the code for Hi-net
# 20 mins data since 2010-01-01T00:00(GMT+0900) from Hi-net
# data,ctable = client.get_continuous_waveform('0101','201103012000',60*24, outdir='Data')
#win32.extract_sac(data,ctable,outdir='Data')
sta={'latitude':37.4703, 'longitude': 140.7227, 'depth':510.0, 'name': 'N.MKJH'}


In [ ]:
cat=pd.read_csv("JMA_total.csv")
cat['time']=pd.to_datetime(cat['time'].values[:])


In [ ]:
folder='/home/tian_feng/bigeq/BP_MF/Hinet/data_conti_raw/2011.02.20/'
data=glob.glob(folder+'*.cnt')[0]
ctable=glob.glob(folder+'*.ch')[0]
win32.extract_sac(data,ctable,outdir='Data')

In [ ]:
files =glob.glob('./Data/*.SAC')
st = read("./Data/*.SAC")

In [ ]:
#Staion name: st[0].meta.station
#Station longitude: st[0].meta.sac['stlo']
# npts: st[0].meta.npts

In [ ]:
events=cat[(cat['time']>datetime(2011, 2, 20, 0))&(cat['time']<datetime(2011, 2, 21, 0))&(cat['MW']>-1)]
events['time']=np.array(events['time']-datetime(2011, 2, 20, 0)).astype(float)/1e9
events.index=range(len(events['time']))
events

In [ ]:
model = TauPyModel(model="iasp91")
length=st[0].meta.npts
sample_rate=st[0].meta.sampling_rate
y_p=[0]*length
y_s=[0]*length
y_o=[0]*length
x=np.arange(length)
for i in range(len(events['time'])):
    rdis = base.calc_vincenty_inverse(events['La'][i],events['Lo'][i],st[0].meta.sac['stla'],st[0].meta.sac['stlo'])
    rdis = base.kilometer2degrees(rdis[0]/1000) # [m] to [degree] 
    P_arrival = model.get_travel_times(source_depth_in_km=events['Depth'][i]/1000,
                                      distance_in_degree=rdis,
                                      phase_list=["P", "pP"])
    S_arrival = model.get_travel_times(source_depth_in_km=events['Depth'][i]/1000,
                                  distance_in_degree=rdis,
                                  phase_list=["S", "s"])
    if len(P_arrival)>0 and len(S_arrival)>0:
        if (events['MW'][i]>3):
            print('Mw:',events['MW'][i],'rdis:',rdis,'travel_time:',P_arrival[0].time,'arrival:',events['time'][i]+P_arrival[0].time)
        plt.plot([events['time'][i],events['time'][i]],[-800,800],'r')
        plt.plot([events['time'][i]+P_arrival[0].time,events['time'][i]+P_arrival[0].time],[-800,800],'y')
        tmp_o=stats.norm((events['time'][i])*sample_rate, 2000).pdf(x)
        tmp_p=stats.norm((events['time'][i]+P_arrival[0].time)*sample_rate, 2000).pdf(x)
        tmp_s=stats.norm((events['time'][i]+S_arrival[0].time)*sample_rate, 2000).pdf(x)

        y_o += tmp_o/max(tmp_o)
        y_p += tmp_p/max(tmp_p)
        y_s += tmp_s/max(tmp_s)


In [ ]:
start=2990000
data=np.zeros([47, 3, 6000], dtype=np.uint8)
for i in range(47):
        for k in range(3):
            data[i][k]=st[i*3+k][start-3000:start+3000]
data=np.swapaxes(data,1,2)
# img.save('my.png')
plt.figure(figsize=(10,20))
plt.subplot(211)

plt.imshow(data)
plt.subplot(212)
plt.plot(st[0][start-3000:start+3000]/5)
plt.show()